In [1]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

DEBUG:__main__:hello


# （2022年1月27日～）組み合わせモデルを含めたMAPE表を作成する

1. ✅コア数と問題サイズを両方振ったうえで出力する
2. 出力したデータにおいて不審な点があったので調査する

## 不審な点

モデルA,モデルB,モデルCがあるとすると、組み合わせモデル「モデルA＆モデルB」および「モデルA＆モデルC」のMAPEがモデルAのMAPEと同じになっている。
これは「モデルA＆モデルX」だけで起きているのではなく「モデルB＆モデルX」,「モデルC＆モデルX」でも起きている。

## 調査方法

1. ✅組み合わせモデルが最適となるはずの関数のリストを作成
2. ✅作成した関数リストに該当する関数のみが記録された生DFを作成
3. ✅作成した生DFを用いて、表を作成
4. 表の作成に用いる関数を本ノートに移動して、printなどを用いながらデバッグ


In [2]:
class ModelMultipleEquationForMultipleRegression(ModelBaseForMultipleRegression):
    # 組み合わせモデル（重回帰分析）

    def __init__(
        self,
        inputDF,
        explanatoryVariableColumnNames,
        responseVariableColumnNames,
        equationDict,
        conditionDictForTest={},
        targetDF=None,
    ):
        super().__init__(
            inputDF,
            explanatoryVariableColumnNames,
            responseVariableColumnNames,
            conditionDictForTest={},
            targetDF=None,
        )
        self.equationDict = equationDict

        print(equationDict)

    def inverterLog10Func(x):
        return 10 ** x

    def ipFunc(x):
        return 1 / x

    def transformDataForModel(self, inputDF):
        # inputDFで与えられたデータをモデルに適した形に変形する
        returnDF = inputDF.copy(deep=True)
        print(f"self.equationDict={self.equationDict}")
        # equationDictのキーをループで回す
        for key in self.equationDict.keys():
            ## equationDictのバリューに合った形で変形を実施
            ### 線形モデルの場合
            if self.equationDict[key] == "lin":
                returnDF[key] = inputDF[key]
            ### 反比例モデルの場合
            elif self.equationDict[key] == "ip":
                returnDF[key] = 1 / inputDF[key]
            ### 対数モデルの場合
            elif self.equationDict[key] == "log":
                returnDF[key] = np.log10(inputDF[key])
            else:
                logger.warning(
                    f"not lin, ip, log what it is?(self.equationDict[key]={self.equationDict[key]})"
                )

        return returnDF

    def setUpDataBeforeCalcLr(self):
        # TODO:transformDataForModel()を使って学習用データを変換

        # モデル構築用データ
        self.dataXForPredict = self.transformDataForModel(self.rawExplanaoryVariable)
        self.dataTForPredict = self.rawResponseVariable
        #         self.dataTForPredict = self.transformDataForModel(
        #             self.rawResponseVariable)
        # テスト用データ
        self.dataXForTest = self.transformDataForModel(
            self.rawExplanaoryVariableForTest
        )
        self.dataTForTest = self.rawResponseVariableForTest

    #         self.dataTForTest = self.transformDataForModel(
    #             self.rawResponseVariableForTest)

    def calcLr(self):
        # 実際にモデルを構築する
        self.lr = LinearRegression()
        self.lr.fit(self.dataXForPredict, self.dataTForPredict)

    def predict(self, inputDF):
        # inputDFのデータから構築されたモデルを使って予測を行う

        # inputDFから説明変数データのみを取得
        inputDFOnlyExplanatoryVariableColumn = inputDF[
            self.explanatoryVariableColumnNames
        ]
        # inputDFで与えられたデータをモデルに適した形に変形する
        transformedInputDF = self.transformDataForModel(
            inputDFOnlyExplanatoryVariableColumn
        )
        # 予測を実行
        result = self.lr.predict(transformedInputDF)

        return result

    pass


def test_ModelMultipleEquationForMultipleRegression():

    # 説明変数
    plotX = np.linspace(10, 20, 10)
    plotY = np.linspace(10, 20, 10)
    plotZ = np.linspace(10, 20, 10)
    # 目的変数
    plotT = 10 * plotX + 15 / plotY + 20 * np.log10(plotZ) + 30

    # DFを作成する
    # カラム名のリスト
    columnNames = ["plotX", "plotY", "plotZ", "plotT"]
    datumForDF = [plotX, plotY, plotZ, plotT]
    inputDFForTest = pd.DataFrame(index=columnNames, data=datumForDF).T
    inputDFForTest["functionName"] = "functionName"

    # 目的変数・説明変数のカラム名のリスト
    # 目的変数のカラム名のリスト
    columnNamesForExp = ["plotX", "plotY", "plotZ"]
    # 説明変数のカラム名のリスト
    columnNamesForRes = ["plotT"]
    # 各説明変数に対する式のリスト
    equationDict = {"plotX": "lin", "plotY": "ip", "plotZ": "log"}

    # 予測をする
    # モデルオブジェクトの作成
    objectModel = ModelMultipleEquationForMultipleRegression(
        inputDF=inputDFForTest,
        explanatoryVariableColumnNames=columnNamesForExp,
        responseVariableColumnNames=columnNamesForRes,
        conditionDictForTest={},
        equationDict=equationDict,
    )
    # モデルの生成の準備
    objectModel.setUpDataBeforeCalcLr()
    # モデルの生成
    objectModel.calcLr()
    # モデルによる予測
    # 入力データDFを作成
    inputDFForPredict = pd.DataFrame(inputDFForTest.head(1))
    predictedNum = objectModel.predict(inputDFForPredict)

    # 相対誤差率でテスト対象のデータが想定通りに動作しているかを判断する
    # 相対誤差率を計算するために実データを取得する
    realNum = plotT[0]
    relativeErrorRate = returnRelativeErrorRate(
        realNum=realNum, predictedNum=predictedNum
    )
    print(
        f"\nrealNum={realNum}, predictedNum={predictedNum}, relativeErrorRate={relativeErrorRate}"
    )
    assert relativeErrorRate < 1

In [3]:
# class Models
# 各モデルのオブジェクトデータを保持している。
# 学習用データ・予測対象データを保持している
# 引数名とその説明
# inputDF：入力データの全てを保持したDF（説明変数・目的変数・ベンチマーク名・関数名を最低限保持している）
# expVarColNames：inputDFの列名の中で、説明変数として用いるカラム名のリスト
# resVarColNames：inputDFの列名の中で、説明変数として用いるカラム名のリスト
# targetDF：inputDFとデータ構成は同じだが、予測対象のデータがセットされている
# modelNames：実施するモデル名を指定できる(["modelLin", "modelIp", "modelLog"])


class Models:
    def __init__(
        self,
        inputDF,
        expVarColNames,
        resVarColNames,
        targetDF=None,
        modelNames=["modelLin", "modelIp", "modelLog"],
    ):
        self.inputDF = inputDF
        self.expVarColNames = expVarColNames
        if len(resVarColNames) > 1:
            warnings.warn("目的変数が複数個存在しています")
        self.resVarColNames = resVarColNames
        self.targetDF = targetDF
        self.functionName = ""
        self.benchmarkName = ""
        self.modelNames = modelNames

        if "modelLin" in self.modelNames:
            self.objectModelLin = ModelLinForMultipleRegression(
                inputDF,
                explanatoryVariableColumnNames=expVarColNames,
                responseVariableColumnNames=resVarColNames,
                targetDF=targetDF,
            )
        if "modelIp" in self.modelNames:
            self.objectModelIp = ModelIpForMultipleRegression(
                inputDF,
                explanatoryVariableColumnNames=expVarColNames,
                responseVariableColumnNames=resVarColNames,
                targetDF=targetDF,
            )
        if "modelLog" in self.modelNames:
            self.objectModelLog = ModelLogForMultipleRegression(
                inputDF,
                explanatoryVariableColumnNames=expVarColNames,
                responseVariableColumnNames=resVarColNames,
                targetDF=targetDF,
            )
        if "modelLinAndIp" in self.modelNames:
            equation_dict = {}
            for expVarElement in expVarColNames:
                if expVarElement == "process":
                    equation_dict[expVarElement] = "lin"
                else:
                    equation_dict[expVarElement] = "ip"
            # logger.debug(f"modelLinAndIp:equation_dict={equation_dict}")
            self.objectModelLinAndIp = ModelMultipleEquationForMultipleRegression(
                inputDF,
                explanatoryVariableColumnNames=expVarColNames,
                responseVariableColumnNames=resVarColNames,
                targetDF=targetDF,
                equationDict=equation_dict,
            )
        if "modelLinAndLog" in self.modelNames:
            equation_dict = {}
            for expVarElement in expVarColNames:
                if expVarElement == "process":
                    equation_dict[expVarElement] = "lin"
                else:
                    equation_dict[expVarElement] = "log"
            # logger.debug(f"modelLinAndLog:equation_dict={equation_dict}")
            self.objectModelLinAndLog = ModelMultipleEquationForMultipleRegression(
                inputDF,
                explanatoryVariableColumnNames=expVarColNames,
                responseVariableColumnNames=resVarColNames,
                targetDF=targetDF,
                equationDict=equation_dict,
            )
        if "modelIpAndLin" in self.modelNames:
            equation_dict = {}
            for expVarElement in expVarColNames:
                if expVarElement == "process":
                    equation_dict[expVarElement] = "ip"
                else:
                    equation_dict[expVarElement] = "lin"
            # logger.debug(f"modelIpAndLin:equation_dict={equation_dict}")
            self.objectModelIpAndLin = ModelMultipleEquationForMultipleRegression(
                inputDF,
                explanatoryVariableColumnNames=expVarColNames,
                responseVariableColumnNames=resVarColNames,
                targetDF=targetDF,
                equationDict=equation_dict,
            )
        if "modelIpAndLog" in self.modelNames:
            equation_dict = {}
            for expVarElement in expVarColNames:
                if expVarElement == "process":
                    equation_dict[expVarElement] = "ip"
                else:
                    equation_dict[expVarElement] = "log"
            # logger.debug(f"modelIpAndLog:equation_dict={equation_dict}")
            self.objectModelIpAndLog = ModelMultipleEquationForMultipleRegression(
                inputDF,
                explanatoryVariableColumnNames=expVarColNames,
                responseVariableColumnNames=resVarColNames,
                targetDF=targetDF,
                equationDict=equation_dict,
            )
        if "modelLogAndLin" in self.modelNames:
            equation_dict = {}
            for expVarElement in expVarColNames:
                if expVarElement == "process":
                    equation_dict[expVarElement] = "log"
                else:
                    equation_dict[expVarElement] = "lin"
            # logger.debug(f"modelLogAndLin:equation_dict={equation_dict}")
            self.objectModelLogAndLin = ModelMultipleEquationForMultipleRegression(
                inputDF,
                explanatoryVariableColumnNames=expVarColNames,
                responseVariableColumnNames=resVarColNames,
                targetDF=targetDF,
                equationDict=equation_dict,
            )
        if "modelLogAndIp" in self.modelNames:
            equation_dict = {}
            for expVarElement in expVarColNames:
                if expVarElement == "process":
                    equation_dict[expVarElement] = "log"
                else:
                    equation_dict[expVarElement] = "ip"
            # logger.debug(f"modelLogAndIp:equation_dict={equation_dict}")
            self.objectModelLogAndIp = ModelMultipleEquationForMultipleRegression(
                inputDF,
                explanatoryVariableColumnNames=expVarColNames,
                responseVariableColumnNames=resVarColNames,
                targetDF=targetDF,
                equationDict=equation_dict,
            )

    def setUpDataBeforeCalcLr(self):
        if "modelLin" in self.modelNames:
            self.objectModelLin.setUpDataBeforeCalcLr()
        if "modelIp" in self.modelNames:
            self.objectModelIp.setUpDataBeforeCalcLr()
        if "modelLog" in self.modelNames:
            self.objectModelLog.setUpDataBeforeCalcLr()
        if "modelLinAndIp" in self.modelNames:
            self.objectModelLinAndIp.setUpDataBeforeCalcLr()
        if "modelLinAndLog" in self.modelNames:
            self.objectModelLinAndLog.setUpDataBeforeCalcLr()
        if "modelIpAndLin" in self.modelNames:
            self.objectModelIpAndLin.setUpDataBeforeCalcLr()
        if "modelIpAndLog" in self.modelNames:
            self.objectModelIpAndLog.setUpDataBeforeCalcLr()
        if "modelLogAndLin" in self.modelNames:
            self.objectModelLogAndLin.setUpDataBeforeCalcLr()
        if "modelLogAndIp" in self.modelNames:
            self.objectModelLogAndIp.setUpDataBeforeCalcLr()

    def calcLr(self):
        if "modelLin" in self.modelNames:
            self.objectModelLin.calcLr()
        if "modelIp" in self.modelNames:
            self.objectModelIp.calcLr()
        if "modelLog" in self.modelNames:
            self.objectModelLog.calcLr()
        if "modelLinAndIp" in self.modelNames:
            self.objectModelLinAndIp.calcLr()
        if "modelLinAndLog" in self.modelNames:
            self.objectModelLinAndLog.calcLr()
        if "modelIpAndLin" in self.modelNames:
            self.objectModelIpAndLin.calcLr()
        if "modelIpAndLog" in self.modelNames:
            self.objectModelIpAndLog.calcLr()
        if "modelLogAndLin" in self.modelNames:
            self.objectModelLogAndLin.calcLr()
        if "modelLogAndIp" in self.modelNames:
            self.objectModelLogAndIp.calcLr()

    # inputDF：__init__()でのinputDFとDF構成は同じ
    def predict(self, inputDF):
        pass

    # 学習用データへの適合度（MAPE[%]）を計算する
    def calcMAPE(self):
        # MAPEatTrain:辞書
        # キーはmodelNamesの要素、バリューは学習データの適合度としてのMAPE
        MAPEatTrain = {}
        if len(self.resVarColNames) > 1:
            warnings.warn("目的変数が複数カラムに及んでいるため、正常な動作を期待できません")
        realData = self.inputDF[self.resVarColNames[0]].tolist()
        if "modelLin" in self.modelNames:
            
            print(f"self.inputDF[self.expVarColNames] = {self.inputDF[self.expVarColNames]}")
            
            print(f"self.objectModelLin.dataXForPredict = {self.objectModelLin.dataXForPredict}")
            
            predictedDataAtLin = self.objectModelLin.predict(
                self.inputDF[self.expVarColNames]
            )
            
            predictedDataAtLin_fromExpVarColNames = self.objectModelLin.predict(
                self.inputDF[self.expVarColNames]
            )
            predictedDataAtLin_fromDataXForPredict = self.objectModelLin.predict(
                self.objectModelLin.dataXForPredict
            )
            
            print(f"predictedDataAtLin_fromDataXForPredict == predictedDataAtLin_fromExpVarColNames={predictedDataAtLin_fromDataXForPredict == predictedDataAtLin_fromExpVarColNames}")
            
            modelLinMAPEatTrain = returnMapeScore(realData, predictedDataAtLin)
            MAPEatTrain["modelLin"] = modelLinMAPEatTrain
        if "modelIp" in self.modelNames:
            predictedDataAtIp = self.objectModelIp.predict(
                self.inputDF[self.expVarColNames]
            )
            modelIpMAPEatTrain = returnMapeScore(realData, predictedDataAtIp)
            MAPEatTrain["modelIp"] = modelIpMAPEatTrain
        if "modelLog" in self.modelNames:
            predictedDataAtLog = self.objectModelLog.predict(
                self.inputDF[self.expVarColNames]
            )
            modelLogMAPEatTrain = returnMapeScore(realData, predictedDataAtLog)
            MAPEatTrain["modelLog"] = modelLogMAPEatTrain
        if "modelLinAndIp" in self.modelNames:
            
            print(f"self.inputDF[self.expVarColNames] = {self.inputDF[self.expVarColNames]}")
            
            print(f"self.objectModelLinAndIp.dataXForPredict = {self.objectModelLinAndIp.dataXForPredict}")
            
            predictedDataAtLinAndIp = self.objectModelLinAndIp.predict(
                self.inputDF[self.expVarColNames]
            )
            modelLinAndIpMAPEatTrain = returnMapeScore(
                realData, predictedDataAtLinAndIp
            )
            MAPEatTrain["modelLinAndIp"] = modelLinAndIpMAPEatTrain
        if "modelLinAndLog" in self.modelNames:
            predictedDataAtLinAndLog = self.objectModelLinAndLog.predict(
                self.inputDF[self.expVarColNames]
            )
            modelLinAndLogMAPEatTrain = returnMapeScore(
                realData, predictedDataAtLinAndLog
            )
            MAPEatTrain["modelLinAndLog"] = modelLinAndLogMAPEatTrain
        if "modelIpAndLin" in self.modelNames:
            predictedDataAtIpAndLin = self.objectModelIpAndLin.predict(
                self.inputDF[self.expVarColNames]
            )
            modelIpAndLinMAPEatTrain = returnMapeScore(
                realData, predictedDataAtIpAndLin
            )
            MAPEatTrain["modelIpAndLin"] = modelIpAndLinMAPEatTrain
        if "modelIpAndLog" in self.modelNames:
            predictedDataAtIpAndLog = self.objectModelIpAndLog.predict(
                self.inputDF[self.expVarColNames]
            )
            modelIpAndLogMAPEatTrain = returnMapeScore(
                realData, predictedDataAtIpAndLog
            )
            MAPEatTrain["modelIpAndLog"] = modelIpAndLogMAPEatTrain
        if "modelLogAndLin" in self.modelNames:
            predictedDataAtLogAndLin = self.objectModelLogAndLin.predict(
                self.inputDF[self.expVarColNames]
            )
            modelLogAndLinMAPEatTrain = returnMapeScore(
                realData, predictedDataAtLogAndLin
            )
            MAPEatTrain["modelLogAndLin"] = modelLogAndLinMAPEatTrain
        if "modelLogAndIp" in self.modelNames:
            predictedDataAtLogAndIp = self.objectModelLogAndIp.predict(
                self.inputDF[self.expVarColNames]
            )
            modelLogAndIpMAPEatTrain = returnMapeScore(
                realData, predictedDataAtLogAndIp
            )
            MAPEatTrain["modelLogAndIp"] = modelLogAndIpMAPEatTrain
        self.MAPEatTrain = MAPEatTrain

    # calcMAPEで計算した辞書を返す関数
    # 返す辞書がない場合は空の辞書を返す
    def returnCalculatedMAPE(self):
        if self.MAPEatTrain is None:
            reutnrn({})
        else:
            return self.MAPEatTrain

    # 予測対象データとの相対誤差率を計算する
    # 引数 targetDF:本オブジェクト構築時に必要になるinputDFをデータ構造が同じDF
    def calcRelativeErrorRate(self, targetDF=None):
        # relativeErrorRateDict:辞書
        # キーはmodelNamesの要素、バリューは絶対相対誤差率
        relativeErrorRateDict = {}
        # （すでに予測対象の説明変数データがある or targetDF is not None）なら問題ない。
        if (self.targetDF is None) and (targetDF is None):
            warnings.warn("相対誤差率を計算するための真値が与えられていません。")
            return -1

        if len(self.resVarColNames) == 0:
            warnings.warn("説明変数のカラム名が複数設定されています")
        # targetDFがNoneの場合
        if targetDF is None:
            if len(self.targetDF) > 1:
                warnings.warn("ターゲットとなるDFに要素が2つ以上含まれています。")
            realData = self.targetDF[self.resVarColNames[0]]
            _targetDF = self.targetDF
        # self.targetDFがNoneの場合
        else:
            if len(targetDF) > 1:
                warnings.warn("ターゲットとなるDFに要素が2つ以上含まれています。")
            realData = targetDF[self.resVarColNames[0]]
            _targetDF = targetDF

        # realData は DataFrame なので、それをリスト化して、最初の要素のみ保持する
        realData = realData.tolist()[0]

        if "modelLin" in self.modelNames:
            predictedData = self.objectModelLin.predict(
                _targetDF[self.expVarColNames]
            ).tolist()[0][0]
            relativeErrorRateDict["modelLin"] = returnRelativeErrorRate(
                realNum=realData, predictedNum=predictedData
            )
        if "modelIp" in self.modelNames:
            predictedData = self.objectModelIp.predict(
                _targetDF[self.expVarColNames]
            ).tolist()[0][0]
            relativeErrorRateDict["modelIp"] = returnRelativeErrorRate(
                realNum=realData, predictedNum=predictedData
            )
        if "modelLog" in self.modelNames:
            predictedData = self.objectModelLog.predict(
                _targetDF[self.expVarColNames]
            ).tolist()[0][0]
            relativeErrorRateDict["modelLog"] = returnRelativeErrorRate(
                realNum=realData, predictedNum=predictedData
            )
        if "modelLinAndIp" in self.modelNames:
            predictedData = self.objectModelLinAndIp.predict(
                _targetDF[self.expVarColNames]
            ).tolist()[0][0]
            relativeErrorRateDict["modelLinAndIp"] = returnRelativeErrorRate(
                realNum=realData, predictedNum=predictedData
            )
        if "modelLinAndLog" in self.modelNames:
            predictedData = self.objectModelLinAndLog.predict(
                _targetDF[self.expVarColNames]
            ).tolist()[0][0]
            relativeErrorRateDict["modelLinAndLog"] = returnRelativeErrorRate(
                realNum=realData, predictedNum=predictedData
            )
        if "modelIpAndLin" in self.modelNames:
            predictedData = self.objectModelIpAndLin.predict(
                _targetDF[self.expVarColNames]
            ).tolist()[0][0]
            relativeErrorRateDict["modelIpAndLin"] = returnRelativeErrorRate(
                realNum=realData, predictedNum=predictedData
            )
        if "modelIpAndLog" in self.modelNames:
            predictedData = self.objectModelIpAndLog.predict(
                _targetDF[self.expVarColNames]
            ).tolist()[0][0]
            relativeErrorRateDict["modelIpAndLog"] = returnRelativeErrorRate(
                realNum=realData, predictedNum=predictedData
            )
        if "modelLogAndLin" in self.modelNames:
            predictedData = self.objectModelLogAndLin.predict(
                _targetDF[self.expVarColNames]
            ).tolist()[0][0]
            relativeErrorRateDict["modelLogAndLin"] = returnRelativeErrorRate(
                realNum=realData, predictedNum=predictedData
            )
        if "modelLogAndIp" in self.modelNames:
            predictedData = self.objectModelLogAndIp.predict(
                _targetDF[self.expVarColNames]
            ).tolist()[0][0]
            relativeErrorRateDict["modelLogAndIp"] = returnRelativeErrorRate(
                realNum=realData, predictedNum=predictedData
            )
        self.relativeErrorRateDict = relativeErrorRateDict

    # calcRelativeErrorRate()で計算した辞書を返す関数
    # 返す辞書がない場合は空の辞書を返す
    def returnRelativeErrorRateDict(self):
        if self.relativeErrorRateDict is None:
            return {}
        else:
            return self.relativeErrorRateDict

    # 関数名・ベンチマーク名を更新する
    def updateFunctionAndBenchmarkName(self, functionName=None, benchmarkName=None):
        # TODO:各種引数が空の場合は更新しない
        if functionName is not None:
            self.functionName = functionName
        if benchmarkName is not None:
            self.benchmarkName = benchmarkName

    def returnFunctionName(self):
        return self.functionName

    def returnBenchmarkName(self):
        return self.benchmarkName

    def returnExpVarDatumDF(self):
        return self.inputDF[self.expVarColNames]

    def returnResVarDatumDF(self):
        return self.inputDF[self.resVarColNames]

    def returnModelsName(self):
        return self.modelNames


#         self.inputDF = inputDF
#         self.expVarColNames = expVarColNames
#         if len(resVarColNames) > 1:
#             warnings.warn("説明変数が複数個存在しています")
#         self.resVarColNames = resVarColNames
#         self.targetDF = targetDF
#         self.functionName = ""
#         self.benchmarkName = ""
#         self.modelNames = modelNames


def test_Models():
    # inputDFを準備
    # 説明変数
    plotX = np.linspace(1, 20, 10)
    plotY = np.linspace(21, 40, 10)
    plotZ = np.linspace(41, 60, 10)
    # 目的変数
    plotTforLin = plotX + 2 * plotY + 3 * plotZ + 4
    plotTforIp = 1 / plotX + 2 / plotY + 3 / plotZ + 4
    plotTforLog = np.log10(plotX) + 2 * np.log10(plotY) + 3 * np.log10(plotZ) + 4
    inputDF = pd.DataFrame(
        {
            "plotX": plotX,
            "plotY": plotY,
            "plotZ": plotZ,
            "plotTforLin": plotTforLin,
            "plotTforIp": plotTforIp,
            "plotTforLog": plotTforLog,
        }
    )

    # functionNameを準備
    functionName = "functionName"
    # benchmarkNameを準備
    benchmarkName = "benchmarkName"

    inputDF[functionName] = functionName
    inputDF[benchmarkName] = benchmarkName
    # targetDFを準備
    targetDF = inputDF.tail(1)

    columnNames = inputDF.columns.tolist()
    # expVarColNamesを準備
    expVarColNames = columnNames[:3]
    # resVarColNamesを準備
    resVarColNames = columnNames[-3:]
    resVarColNamesForLin = ["plotTforLin"]
    resVarColNamesForIp = ["plotTforIp"]
    resVarColNamesForLog = ["plotTforLog"]

    # インスタンスを作成
    modelsLin = Models(
        inputDF=inputDF,
        expVarColNames=expVarColNames,
        resVarColNames=resVarColNamesForLin,
        targetDF=targetDF,
    )
    modelsIp = Models(
        inputDF=inputDF,
        expVarColNames=expVarColNames,
        resVarColNames=resVarColNamesForIp,
        targetDF=targetDF,
    )
    modelsLog = Models(
        inputDF=inputDF,
        expVarColNames=expVarColNames,
        resVarColNames=resVarColNamesForLog,
        targetDF=targetDF,
    )
    # 予測に必要な初期化作業を開始
    modelsLin.setUpDataBeforeCalcLr()
    modelsIp.setUpDataBeforeCalcLr()
    modelsLog.setUpDataBeforeCalcLr()
    # モデル構築を実施
    modelsLin.calcLr()
    modelsIp.calcLr()
    modelsLog.calcLr()

    # 予測をして学習データに対するMAPEを計算し、その値がそれぞれ小さいことを確認
    modelsLin.calcMAPE()
    dictCalcedMAPEatLin = modelsLin.returnCalculatedMAPE()
    modelsIp.calcMAPE()
    dictCalcedMAPEatIp = modelsIp.returnCalculatedMAPE()
    modelsLog.calcMAPE()
    dictCalcedMAPEatLog = modelsLog.returnCalculatedMAPE()
    assert dictCalcedMAPEatLin["modelLin"] < 0.001
    assert dictCalcedMAPEatIp["modelIp"] < 0.001
    assert dictCalcedMAPEatLog["modelLog"] < 0.001

    # 相対誤差率を計算し、それが小さいことを確認
    modelsLin.calcRelativeErrorRate(targetDF=targetDF)
    modelsIp.calcRelativeErrorRate(targetDF=targetDF)
    modelsLog.calcRelativeErrorRate(targetDF=targetDF)
    relativeErrorRateDictAtLin = modelsLin.returnRelativeErrorRateDict()
    relativeErrorRateDictAtIp = modelsIp.returnRelativeErrorRateDict()
    relativeErrorRateDictAtLog = modelsLog.returnRelativeErrorRateDict()
    assert relativeErrorRateDictAtLin["modelLin"] < 0.0001
    assert relativeErrorRateDictAtIp["modelIp"] < 0.0001
    assert relativeErrorRateDictAtLog["modelLog"] < 0.0001

    # 関数名・ベンチマーク名を更新する関数のテスト
    modelsLin.updateFunctionAndBenchmarkName(functionName=functionName)
    modelsIp.updateFunctionAndBenchmarkName(functionName=functionName)
    modelsLog.updateFunctionAndBenchmarkName(functionName=functionName)
    assert (
        modelsLin.returnFunctionName() == functionName
        and modelsIp.returnFunctionName() == functionName
        and modelsLog.returnFunctionName() == functionName
    )
    assert (
        modelsLin.returnBenchmarkName() == ""
        and modelsIp.returnBenchmarkName() == ""
        and modelsLog.returnBenchmarkName() == ""
    )
    modelsLin.updateFunctionAndBenchmarkName(benchmarkName=benchmarkName)
    modelsIp.updateFunctionAndBenchmarkName(benchmarkName=benchmarkName)
    modelsLog.updateFunctionAndBenchmarkName(benchmarkName=benchmarkName)
    assert (
        modelsLin.returnFunctionName() == functionName
        and modelsIp.returnFunctionName() == functionName
        and modelsLog.returnFunctionName() == functionName
    )
    assert (
        modelsLin.returnBenchmarkName() == benchmarkName
        and modelsIp.returnBenchmarkName() == benchmarkName
        and modelsLog.returnBenchmarkName() == benchmarkName
    )

In [4]:
def return_MAPE_Table_DF_from_rawDF(
    rawDF, exp_var_list=[], res_var_list=[], model_name_list=[]
):

    """return_MAPE_Table_DF_from_rawDF()の説明

    引数として渡された生データ入りDFから各モデルでのMAPEを記録したDFを返す関数

    Args:
        rawDF (pandas.DataFrame): DF["functionName<必須列>", "データ列名0<全ての要素は数値>" ... ,"データ列名N<全ての要素は数値>"]
        model_name_list (list): モデル名が要素のリスト
        version(int): バージョン。1がオリジナル。

    Returns:
        pandas.DataFrame: DF[関数名(列名ではなくインデックス), モデル名1, ... , モデル名n]

    Raises:
        ExceptionInResearchLib: 引数のリストが空の場合, 渡された生DFに列名「"functionName"」が存在しない場合に発生

    """

    if exp_var_list == []:
        raise ExceptionInResearchLib("説明変数として利用するカラム名のリストが空です")
    if res_var_list == []:
        raise ExceptionInResearchLib("目的変数として利用するカラム名のリストが空です")
    if model_name_list == []:
        raise ExceptionInResearchLib("構築するモデル名のリストが空です")

    function_names = list(set(rawDF["functionName"].tolist()))
    if len(function_names) == 0:
        raise ExceptionInResearchLib("与えられた生データのDFに関数名がありません")

    # 結果を格納するためのDFを作成
    # resultDF = pd.DataFrame(columns=["functionName"] + model_name_list)
    result_series_list = []

    for function_name in function_names:
        # 関数ごとの生データ
        rawDF_per_function = rawDF[rawDF["functionName"] == function_name]
        # モデルの構築
        models = Models(
            inputDF=rawDF_per_function,
            expVarColNames=exp_var_list,
            resVarColNames=res_var_list,
            targetDF=None,
            modelNames=model_name_list,
        )
        models.setUpDataBeforeCalcLr()
        models.calcLr()
        models.calcMAPE()

        # MAPEの算出
        dictCalcedMAPE = models.returnCalculatedMAPE()

        # 算出されたMAPEの数値を小数第一位までにする
        for key in dictCalcedMAPE.keys():
            # dictCalcedMAPE[key] = int(dictCalcedMAPE[key]*10)/10
            dictCalcedMAPE[key] = float(dictCalcedMAPE[key])

        # 関数ごとの結果に格納
        dict_for_series = copy.deepcopy(dictCalcedMAPE)
        dict_for_series["functionName"] = function_name

        series = pd.Series(dict_for_series)
        # TODO:下記のappend()をconcat()にする
        # resultDF = resultDF.append(series, ignore_index=True)
        result_series_list.append(series)
    resultDF = pd.DataFrame(result_series_list)
    return resultDF


def test_return_MAPE_Table_DF_from_rawDF():
    # 構築するモデルのリスト
    model_name_list = ["modelLin", "modelIp", "modelLog"]
    # モデル構築用のDFを作成
    plotX = np.linspace(10, 20, 10)
    plotY = np.linspace(20, 30, 10)
    plotZ = np.linspace(30, 40, 10)
    function_name_list = ["function_name"] * 10
    benchmark_name_list = ["benchmark_name"] * 10
    exp_var_list = ["plotX", "plotY", "plotZ"]
    res_var_list = ["plotT"]
    plotT = 100 * plotX - 500 * plotY + 0.1 * plotZ + 40
    rawDF = pd.DataFrame(
        data={
            "benchmarkName": benchmark_name_list,
            "functionName": function_name_list,
            "plotT": plotT,
            "plotX": plotX,
            "plotY": plotY,
            "plotZ": plotZ,
        }
    )
    # テスト対象の関数からDFを取得
    returnedDF = return_MAPE_Table_DF_from_rawDF(
        rawDF=rawDF,
        exp_var_list=exp_var_list,
        res_var_list=res_var_list,
        model_name_list=model_name_list,
    )
    # modelLinが最小であることを確認
    assert returnedDF.at[0, "modelLin"] < returnedDF.at[0, "modelIp"]
    assert returnedDF.at[0, "modelLin"] < returnedDF.at[0, "modelLog"]

    # TODO:組み合わせモデル(線形+反比例)
    # 構築するモデルのリスト
    model_name_list = [
        "modelLin",
        "modelIp",
        "modelLog",
        "modelLinAndLog",
        "modelLinAndIp",
    ]
    # モデル構築用のDFを作成
    plotX = np.linspace(10, 20, 10)
    plotY = np.linspace(20, 30, 10)
    plotZ = np.linspace(30, 40, 10)
    function_name_list = ["function_name"] * 10
    benchmark_name_list = ["benchmark_name"] * 10
    exp_var_list = ["process", "plotY", "plotZ"]
    res_var_list = ["plotT"]
    plotT = 100 * plotX + -100 * plotY + 40 / plotZ + 30
    rawDF = pd.DataFrame(
        data={
            "benchmarkName": benchmark_name_list,
            "functionName": function_name_list,
            "plotT": plotT,
            "process": plotX,
            "plotY": plotY,
            "plotZ": plotZ,
        }
    )
    # テスト対象の関数からDFを取得
    returnedDF = return_MAPE_Table_DF_from_rawDF(
        rawDF=rawDF,
        exp_var_list=exp_var_list,
        res_var_list=res_var_list,
        model_name_list=model_name_list,
    )
    # modelLinAndIpが最小であることを確認
    print(returnedDF)
    for model_name in model_name_list:
        if model_name == "modelLinAndIp":
            pass
        else:
            assert returnedDF.at[0, "modelLinAndIp"] < returnedDF.at[0, model_name]

    return

In [5]:
benchmark_name = "cg"

classes = ["A", "B", "C", "D", "E", "F"]
processes = [2, 4, 8, 16, 32, 64, 128, 256]

dict_column_names_JP = {
    "functionName": "関数名",
    "modelLin": "線形",
    "modelIp": "反比例",
    "modelLog": "対数",
    "modelLinAndIp": "線形＆反比例",
    "modelLinAndLog": "線形＆対数",
    "modelIpAndLin": "反比例＆線形",
    "modelIpAndLog": "反比例＆対数",
    "modelLogAndLin": "対数＆線形",
    "modelLogAndIp": "対数＆反比例",
}

model_names_list_JP = list(dict_column_names_JP.values())
model_names_list_JP.remove("関数名")

model_names_list = list(dict_column_names_JP.keys())
model_names_list.remove("functionName")

date = "2022年2月2日"

# コア数と問題サイズの両方を振る

rawDF = return_rawDF_with_init_param(
    benchmark_name = benchmark_name,
    classes= classes,
    processes = processes,
    csv_dir_path = "./csv_files/",
)

# TODO:関数をリスト化
target_function_names = ["CONJ_GRAD", "ICNVRT", "MPI_Irecv()", "SPRNVC", "VECSET"]
# TODO:ここで関数を抽出
target_function = target_function_names[0]
target_function_rawDF = rawDF[rawDF["functionName"] == target_function]

print(target_function_rawDF)

exp_var = target_function_rawDF.columns.tolist()
for element_be_removed in ["functionName", "functionCallNum", "benchmarkName", "benchmarkClass"]:
    exp_var.remove(element_be_removed)
res_var = ["functionCallNum"]

returned_MAPE_table = return_MAPE_Table_DF_from_rawDF(
    rawDF = target_function_rawDF,
    exp_var_list = exp_var,
    res_var_list = res_var,
    model_name_list = model_names_list
)
df = returned_MAPE_table.rename(columns=dict_column_names_JP)
df = df.set_index("関数名")
df = df.sort_index(axis=0)
df = df.sort_index(axis=1)

df = addLowestMAPEsModelNameColumn(df, version=2, model_name_list=model_names_list_JP)
print(
    df
)
# print(
#     df.to_latex(
#         caption=f"ベンチマークプログラム{benchmark_name.upper()}における各モデルでのMAPE",
#         label=f"{date}everyModelsMAPEtableIn{benchmark_name.upper()}",
#     )
# )

  functionName  functionCallNum benchmarkName benchmarkClass  process  \
5    CONJ_GRAD             16.0            cg              A        2   
5    CONJ_GRAD             16.0            cg              A        4   
5    CONJ_GRAD             16.0            cg              A        8   
5    CONJ_GRAD             16.0            cg              A       16   
5    CONJ_GRAD             16.0            cg              A       32   
5    CONJ_GRAD             16.0            cg              A       64   
5    CONJ_GRAD             16.0            cg              A      128   
6    CONJ_GRAD             16.0            cg              A      256   
2    CONJ_GRAD             31.0            cg              B        2   
2    CONJ_GRAD             46.0            cg              B        4   
2    CONJ_GRAD             76.0            cg              B        8   
2    CONJ_GRAD             76.0            cg              B       16   
2    CONJ_GRAD             76.0            cg      

# (～2022年1月27日)ベンチマークCGにおけるすべてのモデルでのMAPE

In [6]:
benchmark_name = "cg"

classes = ["A", "B", "C", "D", "E", "F"]
processes = [2, 4, 8, 16, 32, 64, 128, 256]

dict_column_names_JP = {
    "functionName": "関数名",
    "modelLin": "線形",
    "modelIp": "反比例",
    "modelLog": "対数",
    "modelLinAndIp": "線形＆反比例",
    "modelLinAndLog": "線形＆対数",
    "modelIpAndLin": "反比例＆線形",
    "modelIpAndLog": "反比例＆対数",
    "modelLogAndLin": "対数＆線形",
    "modelLogAndIp": "対数＆反比例",
}

model_names_list_JP = list(dict_column_names_JP.values())
model_names_list_JP.remove("関数名")

model_names_list = list(dict_column_names_JP.keys())
model_names_list.remove("functionName")

date = "2022年2月2日"

# コア数と問題サイズの両方を振る

rawDF = return_rawDF_with_init_param(
    benchmark_name = benchmark_name,
    classes= classes,
    processes = processes,
    csv_dir_path = "./csv_files/",
)

exp_var = rawDF.columns.tolist()
for element_be_removed in ["functionName", "functionCallNum", "benchmarkName", "benchmarkClass"]:
    exp_var.remove(element_be_removed)
res_var = ["functionCallNum"]

returned_MAPE_table = return_MAPE_Table_DF_from_rawDF(
    rawDF = rawDF,
    exp_var_list = exp_var,
    res_var_list = res_var,
    model_name_list = model_names_list
)
df = returned_MAPE_table.rename(columns=dict_column_names_JP)
df = df.set_index("関数名")
df = df.sort_index(axis=0)
df = df.sort_index(axis=1)

df = addLowestMAPEsModelNameColumn(df, version=2, model_name_list=model_names_list_JP)

print(
    df.to_latex(
        caption=f"ベンチマークプログラム{benchmark_name.upper()}における各モデルでのMAPE",
        label=f"{date}everyModelsMAPEtableIn{benchmark_name.upper()}",
    )
)

{'process': 'lin', 'intBenchmarkClass': 'ip', 'na': 'ip', 'nonzer': 'ip', 'niter': 'ip', 'shift': 'ip'}
{'process': 'lin', 'intBenchmarkClass': 'log', 'na': 'log', 'nonzer': 'log', 'niter': 'log', 'shift': 'log'}
{'process': 'ip', 'intBenchmarkClass': 'lin', 'na': 'lin', 'nonzer': 'lin', 'niter': 'lin', 'shift': 'lin'}
{'process': 'ip', 'intBenchmarkClass': 'log', 'na': 'log', 'nonzer': 'log', 'niter': 'log', 'shift': 'log'}
{'process': 'log', 'intBenchmarkClass': 'lin', 'na': 'lin', 'nonzer': 'lin', 'niter': 'lin', 'shift': 'lin'}
{'process': 'log', 'intBenchmarkClass': 'ip', 'na': 'ip', 'nonzer': 'ip', 'niter': 'ip', 'shift': 'ip'}
self.equationDict={'process': 'lin', 'intBenchmarkClass': 'ip', 'na': 'ip', 'nonzer': 'ip', 'niter': 'ip', 'shift': 'ip'}
self.equationDict={'process': 'lin', 'intBenchmarkClass': 'ip', 'na': 'ip', 'nonzer': 'ip', 'niter': 'ip', 'shift': 'ip'}
self.equationDict={'process': 'lin', 'intBenchmarkClass': 'log', 'na': 'log', 'nonzer': 'log', 'niter': 'log', 'sh

/tmp/ipykernel_388/928240918.py:55: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  df.to_latex(


# （2022年1月27日）✅型の相違によって警告が出力されているので修正する
# （2022年1月27日）✅DataFrame.append()がなくなる予定になったために表示される警告が出力されているので修正する

# （2022年1月19日～2022年1月26日）✅最適モデルをまとめた表を作成する処理の確認

`returnDictAboutMAPETable()` の問題に伴い、最適モデルの組み合わせをまとめた表の結果が怪しい。修正する。

In [7]:
benchmark_name = "cg"

fix_class = ["B"]
fix_process = [128]

classes = ["A", "B", "C", "D", "E", "F"]
processes = [2, 4, 8, 16, 32, 64, 128, 256]

dict_column_names_JP = {
    "functionName": "関数名",
    "modelLin": "線形モデル",
    "modelIp": "反比例モデル",
    "modelLog": "対数モデル",
}

model_names_list_JP = list(dict_column_names_JP.values())
model_names_list_JP.remove("関数名")

model_names_list = list(dict_column_names_JP.keys())
model_names_list.remove("functionName")

dict_column_fix_process = {"最適モデル": "最適モデル（コア数固定）"}
dict_column_fix_core = {"最適モデル": "最適モデル（問題サイズ固定）"}

date = "2022年1月22日"

# 問題サイズを固定してコア数を振る
fix_class_rawDF = return_rawDF_with_init_param(
    benchmark_name=benchmark_name,
    classes=fix_class,
    processes=processes,
    csv_dir_path="./csv_files/",
)

exp_var = fix_class_rawDF.columns.tolist()
exp_var.remove("functionName")
exp_var.remove("functionCallNum")
exp_var.remove("benchmarkName")
exp_var.remove("benchmarkClass")
res_var = ["functionCallNum"]

fix_class_MAPE_table = return_MAPE_Table_DF_from_rawDF(
    rawDF=fix_class_rawDF,
    exp_var_list=exp_var,
    res_var_list=res_var,
    model_name_list=model_names_list,
)

# コア数を固定して問題サイズを振る
fix_process_rawDF = return_rawDF_with_init_param(
    benchmark_name=benchmark_name,
    classes=classes,
    processes=fix_process,
    csv_dir_path="./csv_files/",
)

fix_process_MAPE_table = return_MAPE_Table_DF_from_rawDF(
    rawDF=fix_process_rawDF,
    exp_var_list=exp_var,
    res_var_list=res_var,
    model_name_list=model_names_list,
)

# 2条件(問題サイズ固定・コア数固定)で取得したMAPE表に最低MAPE値を記録した列を追加する
fix_process_MAPE_table = addLowestMAPEsModelNameColumn(
    fix_process_MAPE_table, model_name_list=model_names_list, version=2
)
fix_process_MAPE_table = fix_process_MAPE_table.set_index("functionName")
fix_class_MAPE_table = addLowestMAPEsModelNameColumn(
    fix_class_MAPE_table, model_name_list=model_names_list, version=2
)
fix_class_MAPE_table = fix_class_MAPE_table.set_index("functionName")

fix_process_MAPE_table_bestModel = (
    fix_process_MAPE_table.loc[:, ["最適モデル"]]
    .rename(columns={"最適モデル": "最適モデル(コア数固定)"})
    .replace(dict_column_names_JP)
)
fix_core_MAPE_table_bestModel = (
    fix_class_MAPE_table.loc[:, ["最適モデル"]]
    .rename(columns={"最適モデル": "最適モデル(問題サイズ固定)"})
    .replace(dict_column_names_JP)
)

print(
    pd.merge(
        fix_core_MAPE_table_bestModel,
        fix_process_MAPE_table_bestModel,
        how="outer",
        right_index=True,
        left_index=True,
    )
    .sort_index()
    .style.to_latex(
        caption=f"ベンチマークプログラム{benchmark_name.upper()}における最適モデル",
        # index=True,
        label=f"{date}bestModelCombinationIn{benchmark_name.upper()}",
    )
)

self.inputDF[self.expVarColNames] =    process  intBenchmarkClass     na  nonzer  niter  shift
6        2                  4  75000      13     75     60
7        4                  4  75000      13     75     60
5        8                  4  75000      13     75     60
6       16                  4  75000      13     75     60
8       32                  4  75000      13     75     60
7       64                  4  75000      13     75     60
5      128                  4  75000      13     75     60
5      256                  4  75000      13     75     60
self.objectModelLin.dataXForPredict =    process  intBenchmarkClass     na  nonzer  niter  shift
6        2                  4  75000      13     75     60
7        4                  4  75000      13     75     60
5        8                  4  75000      13     75     60
6       16                  4  75000      13     75     60
8       32                  4  75000      13     75     60
7       64                  4  75000     

# （2022年1月19日～）MAPEをまとめた表を作成する関数の修正もしくは別バージョンの作成

`returnDictAboutMAPETable()`は生データを取得する機能とMAPE表を作成する機能がまとまっている。

* ✅生データを取得する機能を持つ関数の作成
    * 引数
        * ベンチマーク名
        * 問題サイズのリスト
        * コア数のリスト
        * 問題サイズごとの初期パラメータの付与の有無
* ✅生データからMAPEを作成する関数の作成
    * 引数
        * 既存の`returnDictAboutMAPETable()`における引数で生データの取得のみに必要な引数は除く

# （2022年1月15日～2022年1月18日）組み合わせモデルの検証

問題サイズ固定時とコア数固定時での最適モデルから予測される組み合わせモデルが、実際に選択される組み合わせモデルと異なる。したがって、その異なっている原因がプログラムの実装ミスに由来するものなのかどうかを検証する。

やること（TODO）

1. ✅問題サイズ固定時とコア数固定時とでは異なるモデルを算出する関数をDFとして抽出
2. ✅各関数でモデルを構築
3. ✅出力した表の列名を英語から日本語にする
4. ✅現行で最適となっているモデルと最適となるはずの組み合わせモデルの違いを探す
    1. returnDictAboutMAPETable()に「ベンチマークプログラムごとに異なる問題サイズ由来の初期変数を追加する処理が含まれていない」問題があったのでそれをやらないといけない

In [8]:
resultsDict = {}

classes = ["A", "B", "C", "D", "E", "F"]
processes = [2, 4, 8, 16, 32, 64, 128]
benchmark_name = "cg"

# ベンチマークプログラム名のリスト
dictColumnNamesJP = {
    "functionName": "関数名",
    "modelLin": "線形",
    "modelLinAndIp": "線形＆反比例",
    "modelLinAndLog": "線形＆対数",
    "modelIp": "反比例",
    "modelIpAndLin": "反比例＆線形",
    "modelIpAndLog": "反比例＆対数",
    "modelLog": "対数",
    "modelLogAndLin": "対数＆線形",
    "modelLogAndIp": "対数＆反比例",
}
model_names_list_jp = list(dictColumnNamesJP.values())
model_names_list_jp.remove("関数名")

model_names_list = list(dictColumnNamesJP.keys())
model_names_list.remove("functionName")
date = "2022年1月15日"

target_function_names = ["CONJ_GRAD", "ICNVRT", "SPRNVC", "VECSET"]

expVar = returnExplanatoryVariablesList(
    benchmarkName=benchmark_name,
    classes=classes,
    processes=processes,
    csvDirPath=csvDirPath,
    baseExpVar=True,
    initExpVar=True,
)

rawDF = returnCollectedExistingData(
    benchmarkNames=[benchmark_name],
    classes=classes,
    processes=processes,
    csvDirPath=csvDirPath,
)

target_functions_rawDF = rawDF[
    (rawDF["functionName"] == target_function_names[0])
    | (rawDF["functionName"] == target_function_names[1])
    | (rawDF["functionName"] == target_function_names[2])
    | (rawDF["functionName"] == target_function_names[3])
]

# 説明変数用に問題サイズを数値化した列を追加
strListProblemSize = target_functions_rawDF["benchmarkClass"].tolist()
intListProblemSize = convertBenchmarkClasses_problemSizeInNPB(
    inputList=strListProblemSize
)
target_functions_rawDF["intBenchmarkClass"] = intListProblemSize
# 説明変数用に問題サイズ由来のほかの数値を保持する列を追加
target_functions_rawDF = addInitDataToRawDF(target_functions_rawDF)

# 目的変数の列名を保持したリスト
res_var = ["functionCallNum"]
# 説明変数の列名を保持したリスト
exp_var = target_functions_rawDF.columns.tolist()
exp_var.remove("functionName")
exp_var.remove("benchmarkName")
exp_var.remove("benchmarkClass")
print(exp_var)

# 結果を格納するためのDFを作成
# resultDF = pd.DataFrame(columns=["functionName"] + model_names_list)
# print(resultDF)
result_series_list = []

for target_function_name in target_function_names:
    # 関数ごとに生データを取得
    target_function_rawDF = target_functions_rawDF[
        target_functions_rawDF["functionName"] == target_function_name
    ]

    # モデルの構築
    models = Models(
        inputDF=target_function_rawDF,
        expVarColNames=exp_var,
        resVarColNames=res_var,
        targetDF=None,
        modelNames=model_names_list,
    )
    models.setUpDataBeforeCalcLr()
    models.calcLr()
    models.calcMAPE()

    # MAPEの算出
    dictCalcedMAPE = models.returnCalculatedMAPE()

    # 算出されたMAPEの数値を小数第一位までにする
    for key in dictCalcedMAPE.keys():
        dictCalcedMAPE[key] = int(dictCalcedMAPE[key] * 10) / 10

    # 関数ごとの結果に格納
    dict_for_series = copy.deepcopy(dictCalcedMAPE)
    dict_for_series["functionName"] = target_function_name

    series = pd.Series(dict_for_series)
    result_series_list.append(series)
    # resultDF = resultDF.append(series, ignore_index=True)
resultDF = pd.DataFrame(result_series_list)
resultDF = resultDF.rename(columns=dictColumnNamesJP)

print(
    resultDF.style.to_latex(
        caption=f"ベンチマークプログラム{benchmark_name.upper()}における特定の関数でのMAPE",
        # index=False,
        label=f"{date}bestModelCombinationIn{benchmark_name.upper()}inTargetFunctions",
    )
)

# df = returnDictAboutMAPETable(
#     benchmarkName=benchmark_name,
#     classes=classes,
#     targetClass=None,
#     processes=processes,
#     targetProcess=None,
#     expVar=expVar,
#     csvDirPath=csvDirPath,
#         modelNames=model_names_list,
# )
# df

/tmp/ipykernel_388/2464667175.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_functions_rawDF["intBenchmarkClass"] = intListProblemSize
/tmp/ipykernel_388/2694063525.py:305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rawDF[dictKey] = columnDict[dictKey]
/tmp/ipykernel_388/2694063525.py:305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyd

['functionCallNum', 'process', 'intBenchmarkClass', 'na', 'nonzer', 'niter', 'shift']
{'functionCallNum': 'ip', 'process': 'lin', 'intBenchmarkClass': 'ip', 'na': 'ip', 'nonzer': 'ip', 'niter': 'ip', 'shift': 'ip'}
{'functionCallNum': 'log', 'process': 'lin', 'intBenchmarkClass': 'log', 'na': 'log', 'nonzer': 'log', 'niter': 'log', 'shift': 'log'}
{'functionCallNum': 'lin', 'process': 'ip', 'intBenchmarkClass': 'lin', 'na': 'lin', 'nonzer': 'lin', 'niter': 'lin', 'shift': 'lin'}
{'functionCallNum': 'log', 'process': 'ip', 'intBenchmarkClass': 'log', 'na': 'log', 'nonzer': 'log', 'niter': 'log', 'shift': 'log'}
{'functionCallNum': 'lin', 'process': 'log', 'intBenchmarkClass': 'lin', 'na': 'lin', 'nonzer': 'lin', 'niter': 'lin', 'shift': 'lin'}
{'functionCallNum': 'ip', 'process': 'log', 'intBenchmarkClass': 'ip', 'na': 'ip', 'nonzer': 'ip', 'niter': 'ip', 'shift': 'ip'}
self.equationDict={'functionCallNum': 'ip', 'process': 'lin', 'intBenchmarkClass': 'ip', 'na': 'ip', 'nonzer': 'ip', '

# （～2022年1月14日）組み合わせモデルにおいて最適なモデルを出力する

| 関数名    | コア数固定での最適モデル | 問題サイズ固定での最適モデル | 
| --------- | ------------------------ | ---------------------------- | 
| func_name | model_name               | model_name                   | 

上記のような表を作成する。

In [9]:
resultsList = []

# コア数を128に固定し、問題サイズを振る
classes_fix_core = ["A", "B", "C", "D", "E", "F"]
processes_fix_core = [128]
# 問題サイズをBに固定し、コア数を振る
classes_fix_class = ["B"]
processes_fix_class = [2, 4, 8, 16, 32, 64, 128]
# ベンチマークプログラム名のリスト
benchmark_names = ["cg"]
# benchmark_names = ["cg", "ep", "ft", "is", "lu", "mg"]
dictColumnNamesJP = {
    "functionName": "関数名",
    "modelLin": "線形モデル",
    "modelIp": "反比例モデル",
    "modelLog": "対数モデル",
}
model_names_list_jp = list(dictColumnNamesJP.values())
model_names_list_jp.remove("関数名")

dict_column_fix_core = {"最適モデル": "最適モデル（コア数固定）"}
dict_column_fix_class = {"最適モデル": "最適モデル（問題サイズ固定）"}

model_names_list = list(dictColumnNamesJP.keys())
model_names_list.remove("functionName")
date = "2022年1月10日"

# 方針

1. ✅「最適モデル」列を追加する関数を作成
2. ✅作成した関数をループ内で使用
3. ✅ループ内で、「コア数を128に固定し、問題サイズを振る」と「問題サイズをBに固定し、コア数を振る」
4. 2つの実験結果を結合させる
    1. ✅それぞれの実験結果から列「最適モデル」を取得
    2. ✅「関数名| 最適モデル(問題サイズ固定)| 最適モデル(コア数固定)」となるように表を操作
5. ✅「関数名｜モデル０｜．．．｜モデルN｜最適モデル」となる表をベンチマークCGで作成
6. ✅「5.」でやったことを他のベンチマークプログラムで実施

In [10]:
# （～2022年1月14日）コア数と問題サイズの両方を振る

classes = ["A", "B", "C", "D", "E", "F"]
processes = [2, 4, 8, 16, 32, 64, 128, 256]

resultDict_fix_core = {}
resultDict_fix_class = {}

# resultDictAllWithoutInit = {}

for benchmarkName in benchmark_names:
    print(f"% _____benchmarkName={benchmarkName}_____\n\n\n")

    # 問題サイズを固定し、コア数を振る
    expVar_fix_class = returnExplanatoryVariablesList(
        benchmarkName=benchmarkName,
        classes=classes_fix_class,
        processes=processes_fix_class,
        csvDirPath=csvDirPath,
        baseExpVar=True,
        initExpVar=True,
    )

    df_fix_class = returnDictAboutMAPETable(
        benchmarkName=benchmarkName,
        classes=classes_fix_class,
        targetClass=None,
        processes=processes_fix_class,
        targetProcess=None,
        expVar=expVar_fix_class,
        csvDirPath=csvDirPath,
    )
    df_fix_class = df_fix_class.rename(columns=dictColumnNamesJP)
    df_fix_class = df_fix_class.set_index("関数名")
    result_fix_class = addLowestMAPEsModelNameColumn(df_fix_class).rename(
        columns=dict_column_fix_class
    )
    resultDict_fix_class[benchmarkName.upper()] = result_fix_class

    # コア数を固定し、問題サイズを振る
    expVar_fix_core = returnExplanatoryVariablesList(
        benchmarkName=benchmarkName,
        classes=classes_fix_core,
        processes=processes_fix_core,
        csvDirPath=csvDirPath,
        baseExpVar=True,
        initExpVar=True,
    )

    df_fix_core = returnDictAboutMAPETable(
        benchmarkName=benchmarkName,
        classes=classes_fix_core,
        targetClass=None,
        processes=processes_fix_core,
        targetProcess=None,
        expVar=expVar_fix_core,
        csvDirPath=csvDirPath,
    )
    df_fix_core = df_fix_core.rename(columns=dictColumnNamesJP)
    df_fix_core = df_fix_core.set_index("関数名")
    result_fix_core = addLowestMAPEsModelNameColumn(df_fix_core).rename(
        columns=dict_column_fix_core
    )
    resultDict_fix_core[benchmarkName.upper()] = result_fix_core

    # 作成した表を結合する
    merged_DF = pd.merge(df_fix_class, df_fix_core)
    dropped_column_name = merged_DF.columns.to_list()
    print(
        pd.merge(
            result_fix_class.drop(model_names_list_jp, axis=1),
            result_fix_core.drop(model_names_list_jp, axis=1),
            how="outer",
            right_index=True,
            left_index=True,
        ).style.to_latex(
            caption=f"ベンチマークプログラム{benchmarkName.upper()}における最適モデル",
            label=f"{date}bestModelCombinationIn{benchmarkName.upper()}",
        )
    )

% _____benchmarkName=cg_____



self.inputDF[self.expVarColNames] =    process  intBenchmarkClass     na  nonzer  niter  shift
0        2                  4  75000      13     75     60
0        4                  4  75000      13     75     60
0        8                  4  75000      13     75     60
0       16                  4  75000      13     75     60
0       32                  4  75000      13     75     60
0       64                  4  75000      13     75     60
0      128                  4  75000      13     75     60
self.objectModelLin.dataXForPredict =    process  intBenchmarkClass     na  nonzer  niter  shift
0        2                  4  75000      13     75     60
0        4                  4  75000      13     75     60
0        8                  4  75000      13     75     60
0       16                  4  75000      13     75     60
0       32                  4  75000      13     75     60
0       64                  4  75000      13     75     60
0      12

predictedDataAtLin_fromDataXForPredict == predictedDataAtLin_fromExpVarColNames=[[ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]]
self.inputDF[self.expVarColNames] =     process  intBenchmarkClass     na  nonzer  niter  shift
18        2                  4  75000      13     75     60
19        4                  4  75000      13     75     60
21        8                  4  75000      13     75     60
21       16                  4  75000      13     75     60
20       32                  4  75000      13     75     60
21       64                  4  75000      13     75     60
21      128                  4  75000      13     75     60
self.objectModelLin.dataXForPredict =     process  intBenchmarkClass     na  nonzer  niter  shift
18        2                  4  75000      13     75     60
19        4                  4  75000      13     75     60
21        8                  4  75000      13     75     60
21       16                  4  75000      13     75     60
20

self.inputDF[self.expVarColNames] =     process  intBenchmarkClass       na  nonzer  niter  shift
11      128                  1    14000      11     15     20
12      128                  4    75000      13     75     60
12      128                 16   150000      15     75    110
13      128                256  1500000      21    100    500
10      128               4096  9000000      26    100   1500
self.objectModelLin.dataXForPredict =     process  intBenchmarkClass       na  nonzer  niter  shift
11      128                  1    14000      11     15     20
12      128                  4    75000      13     75     60
12      128                 16   150000      15     75    110
13      128                256  1500000      21    100    500
10      128               4096  9000000      26    100   1500
predictedDataAtLin_fromDataXForPredict == predictedDataAtLin_fromExpVarColNames=[[ True]
 [ True]
 [ True]
 [ True]
 [ True]]
\begin{table}
\caption{ベンチマークプログラムCGにおける最適モデル}
\label{20

In [11]:
resultsDict = {}

# コア数を128に固定し、問題サイズを振る
classes = ["A", "B", "C", "D", "E", "F"]
processes = [2, 4, 8, 16, 32, 64, 128]
# ベンチマークプログラム名のリスト
# benchmark_names = ["cg", "ep", "ft", "is", "lu", "mg"]
dictColumnNamesJP = {
    "functionName": "関数名",
    "modelLin": "線形",
    "modelIp": "反比例",
    "modelLog": "対数",
    "modelLinAndIp": "線形＆反比例",
    "modelLinAndLog": "線形＆対数",
    "modelIpAndLin": "反比例＆線形",
    "modelIpAndLog": "反比例＆対数",
    "modelLogAndLin": "対数＆線形",
    "modelLogAndIp": "対数＆反比例",
}
model_names_list_jp = list(dictColumnNamesJP.values())
model_names_list_jp.remove("関数名")

model_names_list = list(dictColumnNamesJP.keys())
model_names_list.remove("functionName")
date = "2022年1月10日"

In [12]:
# コア数と問題サイズの両方を振る

for benchmarkName in benchmark_names:
    print(f"% _____benchmarkName={benchmarkName}_____\n\n\n")

    # TODO: 1.「関数名｜モデル０｜．．．｜モデルN｜最適モデル」となる表をベンチマークCGで作成
    # TODO: 2.「1.」でやったことを他のベンチマークプログラムで実施

    expVar = returnExplanatoryVariablesList(
        benchmarkName=benchmarkName,
        classes=classes,
        processes=processes,
        csvDirPath=csvDirPath,
        baseExpVar=True,
        initExpVar=True,
    )
    print(f"expVar={expVar}")

    df = returnDictAboutMAPETable(
        benchmarkName=benchmarkName,
        classes=classes,
        targetClass=None,
        processes=processes,
        targetProcess=None,
        expVar=expVar,
        csvDirPath=csvDirPath,
        modelNames=model_names_list,
    )

    df = df.rename(columns=dictColumnNamesJP)
    df = df.set_index("関数名")
    result = addLowestMAPEsModelNameColumn(df)

    resultsDict[benchmarkName.upper()] = result

    print(
        result.style.to_latex(
            caption=f"ベンチマークプログラム{benchmarkName.upper()}における各モデルでのMAPE",
            label=f"{date}allModelsMAPEin{benchmarkName.upper()}",
        )
    )
#     print(
#         pd.merge(
#             result_fix_class.drop(model_names_list_jp, axis=1),
#             result_fix_core.drop(model_names_list_jp, axis=1),
#             how="outer",
#             right_index=True,
#             left_index=True,
#         ).style.to_latex(caption=f"ベンチマークプログラム{benchmarkName.upper()}における最適モデル", label="bestModelCombinationInCG")
#     )

% _____benchmarkName=cg_____



expVar=['process', 'intBenchmarkClass', 'na', 'nonzer', 'niter', 'shift']
{'process': 'lin', 'intBenchmarkClass': 'ip', 'na': 'ip', 'nonzer': 'ip', 'niter': 'ip', 'shift': 'ip'}
{'process': 'lin', 'intBenchmarkClass': 'log', 'na': 'log', 'nonzer': 'log', 'niter': 'log', 'shift': 'log'}
{'process': 'ip', 'intBenchmarkClass': 'lin', 'na': 'lin', 'nonzer': 'lin', 'niter': 'lin', 'shift': 'lin'}
{'process': 'ip', 'intBenchmarkClass': 'log', 'na': 'log', 'nonzer': 'log', 'niter': 'log', 'shift': 'log'}
{'process': 'log', 'intBenchmarkClass': 'lin', 'na': 'lin', 'nonzer': 'lin', 'niter': 'lin', 'shift': 'lin'}
{'process': 'log', 'intBenchmarkClass': 'ip', 'na': 'ip', 'nonzer': 'ip', 'niter': 'ip', 'shift': 'ip'}
self.equationDict={'process': 'lin', 'intBenchmarkClass': 'ip', 'na': 'ip', 'nonzer': 'ip', 'niter': 'ip', 'shift': 'ip'}
self.equationDict={'process': 'lin', 'intBenchmarkClass': 'ip', 'na': 'ip', 'nonzer': 'ip', 'niter': 'ip', 'shift': 'ip'}
self.equ


self.equationDict={'process': 'lin', 'intBenchmarkClass': 'log', 'na': 'log', 'nonzer': 'log', 'niter': 'log', 'shift': 'log'}
self.equationDict={'process': 'ip', 'intBenchmarkClass': 'lin', 'na': 'lin', 'nonzer': 'lin', 'niter': 'lin', 'shift': 'lin'}
self.equationDict={'process': 'ip', 'intBenchmarkClass': 'log', 'na': 'log', 'nonzer': 'log', 'niter': 'log', 'shift': 'log'}
self.equationDict={'process': 'log', 'intBenchmarkClass': 'lin', 'na': 'lin', 'nonzer': 'lin', 'niter': 'lin', 'shift': 'lin'}
self.equationDict={'process': 'log', 'intBenchmarkClass': 'ip', 'na': 'ip', 'nonzer': 'ip', 'niter': 'ip', 'shift': 'ip'}
{'process': 'lin', 'intBenchmarkClass': 'ip', 'na': 'ip', 'nonzer': 'ip', 'niter': 'ip', 'shift': 'ip'}
{'process': 'lin', 'intBenchmarkClass': 'log', 'na': 'log', 'nonzer': 'log', 'niter': 'log', 'shift': 'log'}
{'process': 'ip', 'intBenchmarkClass': 'lin', 'na': 'lin', 'nonzer': 'lin', 'niter': 'lin', 'shift': 'lin'}
{'process': 'ip', 'intBenchmarkClass': 'log', 'na'